# Counting and measuring cells

Now we'll work through a whole analysis. First we need to load some library code:

In [ ]:
!pip install --quiet dtoolbioimage imageio
from dtoolbioimage import Image

Next we'll load our image:

In [ ]:
from imageio import imread
base_image = imread('sample_cell_image_grayscale.png').view(Image)

Then we can view it:

In [ ]:
base_image

To find the cells, we start by thresholding the image. What happens when we change the parameter?

In [ ]:
base_image > 100

Let's look at a better way to threshold:

In [ ]:
import skimage.filters
block_size=91
threshold = skimage.filters.threshold_local(base_image, block_size)
thresholded = base_image > threshold
thresholded.view(Image)

Next we need to tidy up the result by removing some of the noise:

In [ ]:
from skimage.morphology import remove_small_objects
image = remove_small_objects(thresholded, min_size=1000)
image.view(Image)

Similarly with small light areas:

In [ ]:
from skimage.util import invert
image = invert(image)
image = remove_small_objects(image, min_size=1000)
image = invert(image)
image.view(Image)


Now we need to segment the image:

In [ ]:
from dtoolbioimage.segment import Segmentation
from skimage.measure import label
segmentation = label(image).view(Segmentation)
segmentation

Next we remove the border regions:

In [ ]:
import skimage.segmentation
cleared = skimage.segmentation.clear_border(segmentation)
cleared.view(Segmentation)

Now we'll make a function to annotate our image to label the cells:

In [ ]:
!pip install --quiet jicbioimage.illustrate
from jicbioimage.illustrate import AnnotatedImage
from dtoolbioimage.segment import rprops_by_label

def generate_label_image(segmentation):

    base_for_ann = 100 * (segmentation > 0)
    ann = AnnotatedImage.from_grayscale(base_for_ann)

    for sid, rprops in rprops_by_label(segmentation).items():
        c = rprops.centroid
        ann.text_at(
            str(sid),
            map(int, c),
            size=30,
            color=(255, 255, 0),
            center=True
        )

    return ann.view(Image)

In [ ]:
generate_label_image(cleared)

Finally we can write another function which will actually measure the cell properties:

In [ ]:
from skimage.measure import regionprops
def parameterise_cells(segmentation):

    all_cell_props = regionprops(segmentation)

    cell_information = []
    for props in all_cell_props:

        centroid_row, centroid_col = props.centroid
        cell_entry = dict(
            width=int(props.minor_axis_length),
            length=int(props.major_axis_length),
            area=props.area,
            identifier=props.label,
            perimeter=props.perimeter,
            convex_area=props.convex_area,
            centroid_row=centroid_row,
            centroid_col=centroid_col
        )

        cell_information.append(cell_entry)

    return cell_information

In [ ]:
cell_data = parameterise_cells(cleared)

In [ ]:
!pip install --quiet pandas
import pandas as pd

In [ ]:
pd.DataFrame(cell_data).set_index('identifier')